# Relationship Between Home Purchase Sentiment Index on Housing Prices?

## Rate Hike Rangers (Group 10)

In [1]:
# Import the necessary libraries
import os
from dotenv import load_dotenv
import pandas as pd
import requests
from datetime import datetime


# Load environment variables from .env file
load_dotenv()

True

## FRED Data



In [2]:
# FRED API key
FRED_API_KEY = os.getenv('FRED_API_KEY')


In [3]:
def fetch_fred_data(series_id, start_date=None, end_date=None):
    """
    Fetch data from FRED API
    
    Parameters:
    series_id (str): FRED series identifier
    start_date (str): Start date in 'YYYY-MM-DD' format (optional)
    end_date (str): End date in 'YYYY-MM-DD' format (optional)
    
    Returns:
    pandas.DataFrame: DataFrame with date and value columns
    """
    
    base_url = "https://api.stlouisfed.org/fred/series/observations"
    
    params = {
        'series_id': series_id,
        'api_key': FRED_API_KEY,
        'file_type': 'json'
    }
    
    # Add date filters if provided
    if start_date:
        params['observation_start'] = start_date
    if end_date:
        params['observation_end'] = end_date
    
    # try:
    response = requests.get(base_url, params=params)
    response.raise_for_status()  # Raises an HTTPError for bad responses
    
    data = response.json()
    observations = data['observations']
    
    # Convert to DataFrame
    df = pd.DataFrame(observations)
    
    # Clean up the data
    df['date'] = pd.to_datetime(df['date'])
    df['value'] = pd.to_numeric(df['value'], errors='coerce')
    
    # Remove any rows where value is '.' (FRED's missing data indicator)
    df = df[df['value'].notna()]
    
    # Keep only date and value columns
    df = df[['date', 'value']].reset_index(drop=True)
    
    print(f"Successfully fetched {len(df)} observations for {series_id}")
    return df

In [4]:
# =============================================================================
# UMCSENT DATA 
# =============================================================================
umcsent_data = fetch_fred_data('UMCSENT')

print(f"\nUMCSENT Data Range: {umcsent_data['date'].min()} to {umcsent_data['date'].max()}")
print(f"Latest UMCSENT Value: {umcsent_data['value'].iloc[-1]}")
print(f"Total UMCSENT Observations: {len(umcsent_data)}")
print(umcsent_data.head())



Successfully fetched 661 observations for UMCSENT

UMCSENT Data Range: 1952-11-01 00:00:00 to 2025-05-01 00:00:00
Latest UMCSENT Value: 52.2
Total UMCSENT Observations: 661
        date  value
0 1952-11-01   86.2
1 1953-02-01   90.7
2 1953-08-01   80.8
3 1953-11-01   80.7
4 1954-02-01   82.0


In [5]:
# =============================================================================
# HOME PURCHASE SENTIMENT INDEX (HPSI)
# =============================================================================
hpsi_data = fetch_fred_data('FMNHSHPSIUS')

print(f"\nHPSI Data Range: {hpsi_data['date'].min()} to {hpsi_data['date'].max()}")
print(f"Latest HPSI Value: {hpsi_data['value'].iloc[-1]}")
print(f"Total HPSI Observations: {len(hpsi_data)}")
print(hpsi_data.head())



Successfully fetched 172 observations for FMNHSHPSIUS

HPSI Data Range: 2011-03-01 00:00:00 to 2025-06-01 00:00:00
Latest HPSI Value: 69.8
Total HPSI Observations: 172
        date  value
0 2011-03-01   60.0
1 2011-04-01   62.5
2 2011-05-01   60.3
3 2011-06-01   61.0
4 2011-07-01   60.0


In [6]:
# =============================================================================
# FHFA HOUSE PRICE INDEX (Seasonally Adjusted)
# =============================================================================
hpi_data = fetch_fred_data('HPIPONM226S')

print(f"\nFHFA House Price Index Data Range: {hpi_data['date'].min()} to {hpi_data['date'].max()}")
print(f"Latest FHFA House Price Index Value: {hpi_data['value'].iloc[-1]}")
print(f"Total FHFA House Price Index Observations: {len(hpi_data)}")
print(hpi_data.head())


Successfully fetched 412 observations for HPIPONM226S

FHFA House Price Index Data Range: 1991-01-01 00:00:00 to 2025-04-01 00:00:00
Latest FHFA House Price Index Value: 434.93
Total FHFA House Price Index Observations: 412
        date   value
0 1991-01-01  100.00
1 1991-02-01  100.39
2 1991-03-01  100.46
3 1991-04-01  100.29
4 1991-05-01  100.34


In [7]:
# =============================================================================
# FEDERAL FUNDS RATE
# =============================================================================
fed_rate_data = fetch_fred_data('FEDFUNDS')

print(f"\nFederal Funds Rate Data Range: {fed_rate_data['date'].min()} to {fed_rate_data['date'].max()}")
print(f"Latest Federal Funds Rate Value: {fed_rate_data['value'].iloc[-1]}")
print(f"Total Federal Funds Rate Observations: {len(fed_rate_data)}")
print(fed_rate_data.head())

Successfully fetched 852 observations for FEDFUNDS

Federal Funds Rate Data Range: 1954-07-01 00:00:00 to 2025-06-01 00:00:00
Latest Federal Funds Rate Value: 4.33
Total Federal Funds Rate Observations: 852
        date  value
0 1954-07-01   0.80
1 1954-08-01   1.22
2 1954-09-01   1.07
3 1954-10-01   0.85
4 1954-11-01   0.83


## Fannie Mae Home Price Index

https://www.fanniemae.com/data-and-insights/surveys-indices/fannie-mae-home-price-index

*CSV Downloaded directly from url*


#### How It's Calculated (FHFA Method)


Find Repeat Sales: Identify properties that sold multiple times (e.g., same house sold in 2020 and 2024)
Calculate Price Changes: For each repeat sale, calculate the percentage change between sales
Statistical Modeling: Use regression analysis to estimate price trends while controlling for:

Time between sales
Property characteristics that might have changed
Market conditions


Index Construction: Combine all these individual price changes into a single index number

The Baseline System
FHFA HPI: Uses 1980 Q1 = 100 as the baseline

If today's index = 400, houses cost 4x what they did in 1980 Q1
Your value of ~34 would mean houses cost about 1/3 of 1980 prices (so this is likely very early 1980s data)

In [8]:
#  open the fannie mae home price index csv file

df = pd.read_csv("data_sources/fannie-mae-hpi-sa.csv")
# Display the first few rows of the dataframe
print(df.head())
print(df.tail())

   year  quarter     hpi_sa  rate_of_change
0  1975        1  34.156530             NaN
1  1975        2  34.203690          0.0014
2  1975        3  34.794446          0.0173
3  1975        4  35.379836          0.0168
4  1976        1  35.784635          0.0114
     year  quarter      hpi_sa  rate_of_change
196  2024        1  329.777146          0.0146
197  2024        2  333.403526          0.0110
198  2024        3  336.921984          0.0106
199  2024        4  342.397684          0.0163
200  2025        1  347.123331          0.0138
